In [1]:
# # see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline
# import pdb

In [2]:
import os
import time
import os.path as op
import argparse
from data.data_pipe import get_val_pair
from torchvision import transforms as trans
from tqdm import tqdm_notebook as tqdm
import torch
import numpy as np
from IPython.display import display

from Learner import face_learner
from config import get_config
from utils import hflip_batch, cosineDim1
from data.datasets import (IJBCAllCroppedFacesDataset, IJBCVerificationPathDataset,
                           ARVerificationAllPathDataset, IJBAVerificationDataset)

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
def process_batch_original(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        feat_orig = learner.model.get_original_feature(batch.to(conf.device))
        feat_flip = learner.model.get_original_feature(fliped.to(conf.device))
        feat = (feat_orig + feat_flip) / 2
    else:
        feat = learner.model.get_original_feature(batch.to(conf.device))
    return feat


def process_batch_xcos(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        flattened_feature1, feat_map1 = learner.model(batch.to(conf.device))
        flattened_feature2, feat_map2 = learner.model(fliped.to(conf.device))
        feat_map = (feat_map1 + feat_map2) / 2
        flattened_feature = (flattened_feature1 + flattened_feature2) / 2
    else:
        flattened_feature, feat_map = learner.model(batch.to(conf.device))
    return flattened_feature, feat_map

## Init Learner and conf

In [4]:
conf = get_config(training=False)
conf.batch_size=20 # Why bs_size can only be the number that divide 6000 well?
learner = face_learner(conf, inference=True)

{'fixed_str': 'ir_se50.pth', 'pretrainedMdl': 'ir_se50.pth', 'data_path': PosixPath('data'), 'work_path': PosixPath('work_space'), 'model_path': PosixPath('work_space/models'), 'log_path': PosixPath('work_space/log'), 'save_path': PosixPath('work_space/save'), 'exp_title': 'xCos', 'exp_comment': 'expMS1M', 'input_size': [112, 112], 'embedding_size': 1568, 'use_mobilfacenet': False, 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cuda', index=0), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'emore', 'vgg_folder': PosixPath('data/faces_vgg_112x112'), 'ms1m_folder': PosixPath('data/faces_ms1m_112x112'), 'emore_folder': PosixPath('data/faces_emore'), 'batch_size': 20, 'USE_SOFTMAX': True, 'SOFTMAX_T': 1, 'facebank_path': PosixPath('data/facebank'), 'threshold': 1.5, 'threshold_xCos': 0.2338, 'face_limit': 10, 'min_face_size': 30}
ir_se_50 model generated


# Process IJB-C data and save them

In [ ]:
loader = torch.utils.data.DataLoader(
    IJBCAllCroppedFacesDataset('/tmp3/zhe2325138/IJB/IJB-C/'),
    batch_size=1
)

## Original model

In [ ]:
learner.load_state(conf, 'ir_se50.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)
learner.model.eval()
learner.model.returnGrid = True  # Remember to reset this before return!
learner.model_attention.eval()

dst_dir = './saved_features/IJB-C/ir_se50_original/'
os.makedirs(op.join(dst_dir, 'img'), exist_ok=True)
os.makedirs(op.join(dst_dir, 'frames'), exist_ok=True)

with torch.no_grad():
    for i, batch in enumerate(loader):
        tensor = batch['tensor']
        src_path = batch['path']
        if i % 200 == 0:
            print(f'Processing {src_path[0]}')
        target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npy'
        if op.exists(target_path):
            if i % 200 == 0:
                print(f"Skipping {target_path} because it exists.")
            continue
        feat = process_batch_original(tensor, tta=True)
        np.save(target_path, feat.cpu().numpy())

## Our model

In [22]:
def run_ours_IJBC(model_name):
    learner.load_state(
        conf, f'{model_name}.pth',
        model_only=True, from_save_folder=True, strict=True, model_atten=True)
    learner.model.eval()
    learner.model.returnGrid = True  # Remember to reset this before return!
    learner.model_attention.eval()

    dst_dir = f'./saved_features/IJB-C/{model_name}/'
    os.makedirs(op.join(dst_dir, 'img'), exist_ok=True)
    os.makedirs(op.join(dst_dir, 'frames'), exist_ok=True)

    with torch.no_grad():
        for i, batch in enumerate(loader):
            tensor = batch['tensor']
            src_path = batch['path']
            target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npz'
            if i % 2000 == 0:
                print(f'Processing {src_path[0]} \nSaving to {target_path}')
            if op.exists(target_path):
                if i % 2000 == 0:
                    print(f"Skipping {target_path} because it exists.")
                continue
            flattened_feature, feat_map = process_batch_xcos(tensor, tta=True)
            np.savez(target_path, flattened_feature=flattened_feature.cpu().numpy(),
                     feat_map=feat_map.cpu().numpy())


model_names = [
#     '2019-09-01-15-30_accuracy:0.9946666666666667_step:218346_CosFace',
    '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace',
#     '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None',
    '2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None'
]
for model_name in model_names:
    print(f'-------- Runing on model {model_name} --------')
    run_ours_IJBC(model_name)

-------- Runing on model {model_name} --------
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/10001_101093.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/10001_101093.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/10001_101093.jpg.npz because it exists.
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/12624_114173.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/12624_114173.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/12624_114173.jpg.npz because it exists.
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/13041_117728.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/img/13041_117728.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_ste

Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/13877_177157.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/13877_177157.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/13877_177157.jpg.npz because it exists.
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/14017_204928.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/14017_204928.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/14017_204928.jpg.npz because it exists.
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/143_8206.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/143_8206.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/143_820

Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/2793_396980.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/2793_396980.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/2793_396980.jpg.npz because it exists.
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/28019_396509.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/28019_396509.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/28019_396509.jpg.npz because it exists.
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/28098_387707.jpg 
Saving to ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/28098_387707.jpg.npz
Skipping ./saved_features/IJB-C/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/frames/28

Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/13041_117728.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/img/13041_117728.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/13456_121015.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/img/13456_121015.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/14096_143826.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/img/14096_143826.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/26997_447093.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/img/26997_447093.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/img/27428_452171.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/img/27428_452171.jpg.npz
Processing /tmp3/zhe2325138/IJ

Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/2768_355118.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/frames/2768_355118.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/27754_344557.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/frames/27754_344557.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/27809_453567.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/frames/27809_453567.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/27873_405250.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/frames/27873_405250.jpg.npz
Processing /tmp3/zhe2325138/IJB/IJB-C/cropped_faces/frames/2793_396980.jpg 
Saving to ./saved_features/IJB-C/2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None/frames/2793_396980.jpg.npz
Proc

# IJB-A

In [5]:
loader_IJBA = torch.utils.data.DataLoader(
    IJBAVerificationDataset(only_first_image=False),
    batch_size=1
)
len(loader_IJBA)

11787

In [6]:
def run_ours_IJBA(loader, model_name, xCos_or_original='xCos', only_first_image=True):
    if not only_first_image:
        assert loader.batch_size == 1
        
    model_atten = True if xCos_or_original == 'xCos' else False
    strict = True if xCos_or_original == 'xCos' else False
    learner.load_state(
        conf, f'{model_name}.pth',
        model_only=True, from_save_folder=True, strict=strict, model_atten=model_atten)
    learner.model.eval()
    learner.model.returnGrid = True  # Remember to reset this before return!
    learner.model_attention.eval()

    dst_dir = f'./saved_features/IJB-A_full_template/split1/{model_name}/'
    os.makedirs(dst_dir, exist_ok=True)
    begin_time = time.time()
    with torch.no_grad():
        for i, batch in enumerate(loader):
            if i % 50 == 0:
                print(f"Elapsed {time.time() - begin_time:.1f} seconds.")
            t1_tensors = batch['t1_tensors']
            t2_tensors = batch['t2_tensors']
            
            if not only_first_image:
                t1_tensors, t2_tensors = t2_tensors.squeeze(0), t2_tensors.squeeze(0)
                
            comparison_idx = batch['comparison_idx']
            is_same = batch['is_same']
            
            if xCos_or_original == 'xCos':
                _, feat1 = process_batch_xcos(t1_tensors, tta=True)
                _, feat2 = process_batch_xcos(t2_tensors, tta=True)
            elif xCos_or_original == 'original':
                feat1 = process_batch_original(t1_tensors, tta=True)
                feat2 = process_batch_original(t2_tensors, tta=True)
            else:
                raise NotImplementedError
                
            if not only_first_image:
                feat1, feat2 = feat1.unsqueeze(0), feat2.unsqueeze(0
                                                                  )
#             print(feat1.shape, t1_tensors.shape)

            for idx, f1, f2, same in zip(comparison_idx, feat1, feat2, is_same):
                target_path = op.join(dst_dir, str(int(idx.cpu().numpy()))) + '.npz'
                if op.exists(target_path):
                    if i % 50 == 0:
                        print(f"Skipping {target_path} because it exists.")
                    continue
                if i % 50 == 0:
                    print(f'Saving to {target_path}')
                    
                np.savez(target_path, idx=idx.cpu().numpy(), same=same.cpu().numpy(),
                         f1=f1.cpu().numpy(), f2=f2.cpu().numpy(), )

In [ ]:
run_ours_IJBA(loader_IJBA, 'ir_se50', xCos_or_original='original', only_first_image=False)

Elapsed 2.9 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/0.npz because it exists.
Elapsed 79.4 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/50.npz because it exists.
Elapsed 150.7 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/100.npz because it exists.
Elapsed 326.8 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/150.npz because it exists.
Elapsed 432.5 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/200.npz because it exists.
Elapsed 665.6 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/250.npz because it exists.
Elapsed 754.9 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/300.npz because it exists.
Elapsed 815.4 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/350.npz because it exists.
Elapsed 1039.0 seconds.
Skipping ./saved_features/IJB-A_full_template/split1/ir_se50/400.npz because it exists.
Elapse

/home/master/07/zhe2325138/miniconda3/envs/xCos/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 206 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/master/07/zhe2325138/miniconda3/envs/xCos/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 724 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/master/07/zhe2325138/miniconda3/envs/xCos/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/master/07/zhe2325138/miniconda3/envs/xCos/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 170 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(dat

Elapsed 2155.0 seconds.
Saving to ./saved_features/IJB-A_full_template/split1/ir_se50/900.npz
Elapsed 2373.4 seconds.
Saving to ./saved_features/IJB-A_full_template/split1/ir_se50/950.npz


In [19]:
model_names = [
    '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace',
    '2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None',
#     '2019-09-01-15-30_accuracy:0.9946666666666667_step:218346_CosFace',
#     '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None',
]
for model_name in model_names:
    print(f'-------- Runing on model {model_name} --------')
    run_ours_IJBA(loader_IJBA, model_name, xCos_or_original='xCos', only_first_image=False)

-------- Runing on model 2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace --------
Loading attention module from work_space/save/model_attention_2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace.pth
Skipping ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/0.npz because it exists.
Skipping ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/1.npz because it exists.
Skipping ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/2.npz because it exists.
Skipping ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/3.npz because it exists.
Skipping ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4.npz because it exists.
Skipping ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/5.npz because it exis

Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4500.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4501.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4502.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4503.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4504.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4505.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4506.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/4507.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_ste

Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9000.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9001.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9002.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9003.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9004.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9005.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9006.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/9007.npz
Saving to ./saved_features/IJB-A/split1/2019-09-02-08-21_accuracy:0.9968333333333333_ste

KeyboardInterrupt: 

# Process ARFace data and save them

In [11]:
loader = torch.utils.data.DataLoader(ARVerificationAllPathDataset(), batch_size=1)

In [13]:
learner.load_state(conf, 'ir_se50.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)
learner.model.eval()
learner.model.returnGrid = True  # Remember to reset this before return!
learner.model_attention.eval()

dst_dir = './saved_features/ARFace/ir_se50_original/'
os.makedirs(dst_dir, exist_ok=True)
with torch.no_grad():
    for i, batch in enumerate(loader):
        tensor = batch['image_tensor']
        fname = batch['fname'][0]
        target_path = op.join(dst_dir, fname) + '.npy'
        if i % 200 == 0:
            print(f'Processing {fname}, saving to {target_path}')
        if op.exists(target_path):
            if i % 200 == 0:
                print(f"Skipping {target_path} because it exists.")
            continue
        feat = process_batch_original(tensor, tta=True)
        np.save(target_path, feat.cpu().numpy())

Processing m-001-1, saving to ./saved_features/ARFace/ir_se50_original/m-001-1.npy
Skipping ./saved_features/ARFace/ir_se50_original/m-001-1.npy because it exists.
Processing m-008-4, saving to ./saved_features/ARFace/ir_se50_original/m-008-4.npy
Skipping ./saved_features/ARFace/ir_se50_original/m-008-4.npy because it exists.
Processing m-017-13, saving to ./saved_features/ARFace/ir_se50_original/m-017-13.npy
Skipping ./saved_features/ARFace/ir_se50_original/m-017-13.npy because it exists.
Processing m-025-18, saving to ./saved_features/ARFace/ir_se50_original/m-025-18.npy
Skipping ./saved_features/ARFace/ir_se50_original/m-025-18.npy because it exists.
Processing m-034-15, saving to ./saved_features/ARFace/ir_se50_original/m-034-15.npy
Skipping ./saved_features/ARFace/ir_se50_original/m-034-15.npy because it exists.
Processing m-042-21, saving to ./saved_features/ARFace/ir_se50_original/m-042-21.npy
Skipping ./saved_features/ARFace/ir_se50_original/m-042-21.npy because it exists.
Proc

In [14]:
def compute_our_model(name):
    learner.load_state(conf, f'{name}.pth', model_only=True, from_save_folder=True, strict=True, model_atten=True)
    learner.model.eval()
    learner.model.returnGrid = True  # Remember to reset this before return!
    learner.model_attention.eval()

    dst_dir = f'./saved_features/ARFace/{name}/'
    os.makedirs(dst_dir, exist_ok=True)
    with torch.no_grad():
        for i, batch in enumerate(loader):
            tensor = batch['image_tensor']
            fname = batch['fname'][0]
            target_path = op.join(dst_dir, fname) + '.npz'
            if i % 200 == 0:
                print(f'Processing {fname}, saving to {target_path}')
            if op.exists(target_path):
                if i % 200 == 0:
                    print(f"Skipping {target_path} because it exists.")
                continue
            flattened_feature, feat_map = process_batch_xcos(tensor, tta=True)
            np.savez(target_path, flattened_feature=flattened_feature.cpu().numpy(),
                     feat_map=feat_map.cpu().numpy())
            
names = [
    '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace',
    '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None',
    '2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None',
#     '2019-09-01-15-30_accuracy:0.9946666666666667_step:218346_CosFace',
]
for name in names:
    print(f'---------------------- Testing model {name} ----------------------')
    compute_our_model(name)

---------------------- Testing model 2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace ----------------------
Processing m-001-1, saving to ./saved_features/ARFace/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/m-001-1.npz
Skipping ./saved_features/ARFace/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/m-001-1.npz because it exists.
Processing m-008-4, saving to ./saved_features/ARFace/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/m-008-4.npz
Skipping ./saved_features/ARFace/2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace/m-008-4.npz because it exists.


KeyboardInterrupt: 

In [ ]:
1